In [ ]:
import numpy as np
import h5py as h5
import scqubits as scq
import pandas as pd
import plotly.graph_objects as go

# Load

In [ ]:
qub_name = "S001"
loadpath = "../result/S001/params.csv"
df = pd.read_csv(loadpath)
df = df.set_index("Qubit")
cflx = float(df.loc[qub_name, "Half flux"])*1e3 # mA
period = float(df.loc[qub_name, "Period"])*1e3  # mA
params = (df.loc[qub_name, "EJ"], df.loc[qub_name, "EC"], df.loc[qub_name, "EL"])

eflx = cflx + period / 2 # mA

def phi2mA(ratio):
    return 2*(eflx - cflx)*(ratio - 0.5) + cflx
cflx, eflx

In [ ]:
flxs = np.linspace(0.4, 1.0, 1000)
mAs = phi2mA(flxs)
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=40, truncated_dim=10)
# fluxonium = scq.Fluxonium(8.51,0.91,1.07, flux=0.5, cutoff=40, truncated_dim=10)

# Energy & Transitions

In [ ]:
# loading points
r_f = 7.4876

df = pd.read_csv("../result/S001/freqs.csv", usecols=['Current (mA)', 'Freq (MHz)', 'T1 (μs)'])
s_mAs = df['Current (mA)'].values # mA
s_freqs = df['Freq (MHz)'].values*1e-3 # GHz
s_T1s = df['T1 (μs)'].values

In [ ]:
load_path = "../result/S001/spect.hdf5"
spectrums = []
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        spectrums.append([
            grp["spectrum"]["flxs"][()],
            grp["spectrum"]["fpts"][()],
            grp["spectrum"]["data"][()],
            grp["cflx"][()]
        ])

target_cflx = spectrums[-1][-1]
for spect in spectrums:
    shift = target_cflx - spect[-1]
    spect[0] = phi2mA(spect[0] + shift)
    spect[-1] = target_cflx

In [ ]:
spect = fluxonium.get_spectrum_vs_paramvals("flux", flxs, evals_count=5)
energies = spect.energy_table

In [ ]:
fig = go.Figure()
# spectrum
for flxs_i, fpts_i, data, _ in spectrums:
    data = np.abs(data - np.mean(data, axis=0, keepdims=True))
    data /= np.std(data, axis=0, keepdims=True)
    fig.add_trace(go.Heatmap(z=data**2, x=flxs_i, y=fpts_i, colorscale='Viridis', showscale=False))

all_trans = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)]

# transition
for i, j in [(0, 1)]:
    fig.add_trace(go.Scatter(x=mAs, y=energies[:, j]-energies[:, i], mode='lines', name=f"{i}-{j}"))

# 2 photon transition
for i, j in [(0, 3)]:
    freq = energies[:, j]-energies[:, i]
    fig.add_trace(go.Scatter(x=mAs, y=0.5*freq, mode='lines', name=f"{i}-{j} 2 photon"))

# 2 photon blue sideband
for i, j in [(0, 1)]:
    freq = energies[:, j]-energies[:, i]
    fig.add_trace(go.Scatter(x=mAs, y=0.5*(r_f+freq), mode='lines', name=f"{i}-{j} blue side 2 photon"))

# red sideband
for i, j in [(0, 1)]:
    freq = energies[:, j]-energies[:, i]
    y = r_f-freq
    y[y < 0] = np.nan
    fig.add_trace(go.Scatter(x=mAs, y=y, mode='lines', name=f"{i}-{j} red side"))

# 2 photon red sideband
for i, j in []:
    freq = energies[:, j]-energies[:, i]
    y = 0.5*(r_f-freq)
    y[y < 0] = np.nan
    fig.add_trace(go.Scatter(x=mAs, y=y, mode='lines', name=f"{i}-{j} red side 2 photon"))


# fig.add_trace(go.Scatter(x=s_mAs, y=s_freqs, mode='markers', name="sample", marker=dict(color=-np.array(s_T1s), colorscale='Greys')))
fig.add_hline(y=r_f, line_dash="dash", line_color="black", name="Cavity freq")
fig.add_vline(x=cflx, line_dash="dash", line_color="black", name="Cavity freq")
fig.update_layout(xaxis_title="mA", yaxis_title="Frequency (GHz)")
fig.update_layout(title_text=f"Near transition EJ/EC/EL = {params[0]:.2f}/{params[1]:.2f}/{params[2]:.2f}", title_x=0.5)
fig.update_layout(legend_title_text='Transition')
# fig.update_yaxes(range=[0, max(s_freqs)+0.1])
fig.show()